# Basic-Fit feedbacks analysis

In [49]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.core.os_manager import ChromeType
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd

In [50]:
#set up configurations
options = Options()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--headless")  # Required for Codespaces
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

#read data from csv
df = pd.read_csv("basic-fit.csv")

In [51]:
#create dataframe

df_feedbacks = pd.DataFrame(
    {
        "Author": pd.Series(dtype="str"),
        "Stars": pd.Series(dtype="int"),
        "Date": pd.Series(dtype="str"),
        "Comment": pd.Series(dtype="str"),
        "Link" : pd.Series(dtype="str")
    }
)

In [52]:
#create a list of links
links_list = df['Link'].tolist()

In [ ]:
driver = webdriver.Chrome(options=options)
#driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
actions = ActionChains(driver)

#open the first link from the list
driver.get(links_list[40])

#acceptance offerta
accept_button = driver.find_element("xpath", "//button[@aria-label='Accept all']")
time.sleep(1)
accept_button.click()

time.sleep(3)
#expand comments section
button_more_rev_list = driver.find_elements("xpath", "//button[contains(@aria-label,'More reviews')]")


if button_more_rev_list:
    actions.move_to_element(button_more_rev_list[0]).perform()
    button_more_rev = WebDriverWait(driver,20).until(EC.element_to_be_clickable(("xpath", "//button[contains(@aria-label,'More reviews')]")))

#find review
reviews = int(
   button_more_rev
   .get_attribute("aria-label")
   .split("reviews")[1]
   .strip()[1:-1]   #remove brackets
   .replace(",", "")        #replace comma
   )


button_more_rev.click()

#creat list of id comments
id_list = []

#counter
count = 0

#find all
while(count < reviews):

   xpath = "//div[@class='jftiEf fontBodyMedium'"
   for i in id_list:
       xpath = xpath + f" and not(contains(@data-review-id, '{i}'))"

   xpath = xpath + "]"
    
   #driver.save_screenshot("debug.png")
   comment_section = WebDriverWait(driver,10).until(EC.visibility_of_element_located(("xpath", xpath)))
   
   #scroll down to comment
   comment_section = WebDriverWait(driver,10).until(EC.visibility_of_element_located(("xpath", xpath)))
   actions.move_to_element(comment_section).perform()

   time.sleep(1)

   #scroll down to like panel
   comment_section = WebDriverWait(driver,10).until(EC.visibility_of_element_located(("xpath", xpath)))
   panels = comment_section.find_elements("xpath",(".//div[@class='Gd9ibe OyjIsf']"))
   
   if panels:
        panel = panels[0]
        actions.move_to_element(panel).perform()
        #find panel location
        panel_x = panel.location_once_scrolled_into_view["x"]
        panel_y = panel.location_once_scrolled_into_view["y"]
        #scrol down 500 pixel from the panel
        actions.scroll(panel_x,panel_y, 0, 500).perform()
   else:
       #find stars section location
       stars_section_x = stars_section.location_once_scrolled_into_view["x"]
       stars_section_y = stars_section.location_once_scrolled_into_view["y"]
       #scrol down 500 pixel from the panel
       actions.scroll(stars_section_x, stars_section_y, 0, 500).perform()
       
    
   time.sleep(3)

   #find id
   comment_section =WebDriverWait(driver,10).until(EC.visibility_of_element_located(("xpath", xpath)))
   id = comment_section.get_attribute("data-review-id")
   
   #add id to hte list
   id_list.append(id)
   
   #find comment author
   comment_section =WebDriverWait(driver,10).until(EC.visibility_of_element_located(("xpath", xpath)))
   author_name = comment_section.find_element("xpath", ".//div[@class='d4r55']/span").get_attribute("innerHTML")
   
   #find comment rating
   comment_section =WebDriverWait(driver,10).until(EC.visibility_of_element_located(("xpath", xpath)))
   stars_section = comment_section.find_element("xpath", ".//div[@class='DU9Pgb']/span[2]")
   stars = stars_section.get_attribute("aria-label").strip().split(" ")[0]


   #find publication date
   comment_section =WebDriverWait(driver,10).until(EC.visibility_of_element_located(("xpath", xpath)))
   publication_date = comment_section.find_element("xpath", ".//div[@class='DU9Pgb']/span[3]").get_attribute("innerHTML")
   
   
   #find more button
   comment_section =WebDriverWait(driver,10).until(EC.visibility_of_element_located(("xpath", xpath)))
   more_button = comment_section.find_elements("xpath", ".//button[@class='w8nwRe kyuRq']")
   
   #click on button
   if more_button:
        actions.move_to_element(more_button[0]).perform()
        more_button[0].click()
   
   #find comment
   comment_section =WebDriverWait(driver,10).until(EC.visibility_of_element_located(("xpath", xpath)))
   comments = comment_section.find_elements("xpath", ".//span[@class='wiI7pd']")

   if comments:
       comment = comments[0].get_attribute("innerHTML")
   else:
       comment = None
   
   #create a new row
   new_row = {
        "Author" : f"{author_name}",
        "Stars" : f"{stars}",
        "Date" : f"{publication_date}",
        "Comment" : f"{comment}",
        "Link" : f"{links_list[40]}"
    }
    
   #add the row to the dataframe
   df_feedbacks = pd.concat([df_feedbacks, pd.DataFrame([new_row])], ignore_index=True)
   count += 1


TimeoutException: Message: 
Stacktrace:
#0 0x5a6db8666d0a <unknown>
#1 0x5a6db81175f0 <unknown>
#2 0x5a6db8168a33 <unknown>
#3 0x5a6db8168c21 <unknown>
#4 0x5a6db81b7274 <unknown>
#5 0x5a6db818e68d <unknown>
#6 0x5a6db81b4660 <unknown>
#7 0x5a6db818e433 <unknown>
#8 0x5a6db815aea3 <unknown>
#9 0x5a6db815bb01 <unknown>
#10 0x5a6db862bb5b <unknown>
#11 0x5a6db862fa41 <unknown>
#12 0x5a6db8612c52 <unknown>
#13 0x5a6db86305b4 <unknown>
#14 0x5a6db85f6f0f <unknown>
#15 0x5a6db8654db8 <unknown>
#16 0x5a6db8654f96 <unknown>
#17 0x5a6db8665b56 <unknown>
#18 0x7bacddf7f609 start_thread


In [ ]:
df_feedbacks.shape

(105, 5)

In [ ]:
df_feedbacks.nunique()

Author     105
Stars        3
Date         5
Comment     94
Link         1
dtype: int64

In [ ]:
df_feedbacks.head(5)

,Author,Stars,Date,Comment,Link
0,Marta Martinez,5,a month ago,El gimnasio está genial hay un montón de máqui...,https://www.google.com/maps/place/Basic-Fit+Na...
1,Tania López,5,a month ago,Muy buena maquinaria y muy buen ambiente para ...,https://www.google.com/maps/place/Basic-Fit+Na...
2,Juan Trillo Pita da Veiga,5,a month ago,Es un gimnasio muy grande que dispone de todo ...,https://www.google.com/maps/place/Basic-Fit+Na...
3,Paula Beatriz,5,2 weeks ago,"Muy buen ambiente, personal siempre dispuesto ...",https://www.google.com/maps/place/Basic-Fit+Na...
4,Adriana Sabio,5,2 weeks ago,El gimnasio está genial! Fui con el pase de un...,https://www.google.com/maps/place/Basic-Fit+Na...
